In [1]:
import os
import random
import numpy as np
import torch

In [2]:
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
import torchvision.datasets as datasets
from torchvision import transforms

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

train_dataset = datasets.MNIST(
    root='./',
    download=True,
    train=True,
    transform=transform
)

test_dataset = datasets.MNIST(
    root='./',
    download=False,
    train=False,
    transform= transform
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10_data/cifar-10-python.tar.gz to ./cifar10_data
Files already downloaded and verified


In [6]:
from torch.utils.data.sampler import SubsetRandomSampler

In [8]:
indices = list(range(len(train_dataset)))
np.random.shuffle(indices)
print(indices)
batch_size = 32

[33553, 9427, 199, 12447, 39489, 42724, 10822, 49498, 4144, 36958, 43106, 38695, 6188, 1414, 18471, 29282, 15177, 34304, 12609, 12144, 6113, 15908, 821, 15118, 13466, 26497, 42111, 30188, 37237, 33109, 36480, 24148, 5503, 3918, 38478, 21123, 49717, 42294, 47609, 11076, 41514, 22062, 9413, 38340, 30263, 41252, 14644, 26335, 28102, 17523, 24712, 40485, 3850, 40040, 15267, 49146, 30001, 45926, 43824, 17240, 4892, 725, 22949, 34657, 37213, 20304, 15269, 22367, 42684, 46734, 11732, 18619, 48492, 26803, 44305, 47522, 32207, 2530, 17746, 8270, 15079, 13624, 30638, 30440, 36056, 42651, 36803, 48224, 29226, 5633, 5987, 48063, 1812, 27578, 16823, 3928, 37286, 25045, 15727, 29600, 10430, 25768, 29103, 36537, 39328, 35539, 28075, 17840, 27802, 7682, 24279, 33070, 35557, 37189, 47149, 46915, 9801, 38800, 24669, 44233, 31919, 7134, 33704, 29660, 17893, 22018, 13538, 43118, 36420, 9405, 13413, 6720, 15601, 44743, 31138, 2729, 38242, 29625, 47894, 42090, 36399, 6244, 5545, 4064, 12851, 21802, 41163, 6

In [9]:
split = int(np.floor(0.9 * len(train_dataset)))
tr_idx, val_idx = indices[:split], indices[split:]

In [10]:
tr_sampler = SubsetRandomSampler(tr_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [11]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    num_workers=0,
    sampler=tr_sampler
)
val_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    num_workers=0,
    sampler=val_sampler
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=0
)

In [12]:
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1)
        self.fc1 = nn.Linear(1152, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        batch = len(x)
        x = self.conv1(x)
        x = F.relu(x)
        x = F.avg_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(batch, -1)
        x = self.fc1(x)
        x = F.relu(x)
        logits = self.fc2(x)
        return logits

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
model = Net()
model = model.to(device)

In [16]:
import torch.optim as optim

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [18]:
epochs = 20

for epoch in range(epochs):

    train_loss = 0
    train_total = 0
    train_correct = 0
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
    train_acc = train_correct / train_total

    valid_loss = 0
    valid_total = 0
    valid_correct = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()
    valid_acc = valid_correct / valid_total

    if (epoch+1) % (epochs/10) == 0:
        #print('Epoch: ', epoch + 1)
        #print('Loss: ', 'Train %.3f / Valid %.3f' % (train_loss, valid_loss))
        #print('Accuracy: ', 'Train %.2f / Valid %.2f' % (train_acc * 100, valid_acc * 100))
        print("[%d/%d] TrainLoss: %.3f, ValLoss: %.3f | TrainAcc: %.2f, ValAcc: %.2f’" % (epoch, epochs, train_loss, valid_loss, train_acc, valid_acc))

RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), './2016311076_이동혁.pt')

In [ ]:
new_model = Net()
new_model.load_state_dict(torch.load('./2016311076_이동혁.pt'))

<All keys matched successfully>

In [ ]:
with torch.no_grad():
    test_total = 0
    test_correct = 0
    model.eval()
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()
    test_acc = train_correct / train_total
    print('TestAcc: %.2f' % (test_acc))
    

TestAcc: 0.99
